In [36]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [37]:
# Load the dataset
data = pd.read_csv('../data/MachineLearningRating_v3.txt')
data.head()

C:\Users\pc\AppData\Local\Temp\ipykernel_16280\4101762830.py:2: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/MachineLearningRating_v3.txt')


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [38]:
print(data.isnull().sum())

UnderwrittenCoverID               0
PolicyID                          0
TransactionMonth                  0
IsVATRegistered                   0
Citizenship                       0
LegalType                         0
Title                             0
Language                          0
Bank                              0
AccountType                       0
MaritalStatus                     0
Gender                            0
Country                           0
Province                          0
PostalCode                        0
MainCrestaZone                    0
SubCrestaZone                     0
ItemType                          0
mmcode                            0
VehicleType                     552
RegistrationYear                  0
make                            552
Model                           552
Cylinders                         0
cubiccapacity                     0
kilowatts                         0
bodytype                        552
NumberOfDoors               

In [39]:
# Impute missing values with mode for categorical columns
data['VehicleType'].fillna(data['VehicleType'].mode()[0], inplace=True)
data['make'].fillna(data['make'].mode()[0], inplace=True)
data['Model'].fillna(data['Model'].mode()[0], inplace=True)

C:\Users\pc\AppData\Local\Temp\ipykernel_16280\3990363140.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['VehicleType'].fillna(data['VehicleType'].mode()[0], inplace=True)
C:\Users\pc\AppData\Local\Temp\ipykernel_16280\3990363140.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as

In [40]:
print(data.isnull().sum())

UnderwrittenCoverID               0
PolicyID                          0
TransactionMonth                  0
IsVATRegistered                   0
Citizenship                       0
LegalType                         0
Title                             0
Language                          0
Bank                              0
AccountType                       0
MaritalStatus                     0
Gender                            0
Country                           0
Province                          0
PostalCode                        0
MainCrestaZone                    0
SubCrestaZone                     0
ItemType                          0
mmcode                            0
VehicleType                       0
RegistrationYear                  0
make                              0
Model                             0
Cylinders                         0
cubiccapacity                     0
kilowatts                         0
bodytype                        552
NumberOfDoors               

In [41]:
# Convert TransactionMonth to datetime
data['TransactionMonth'] = pd.to_datetime(data['TransactionMonth'], errors='coerce')

# Optional: Extract year, month, and day as new features
data['TransactionYear'] = data['TransactionMonth'].dt.year
data['TransactionMonthValue'] = data['TransactionMonth'].dt.month

# Create vehicle age feature
data['TransactionYear'] = pd.to_datetime(data['TransactionMonth']).dt.year
data['VehicleAge'] = data['TransactionYear'] - data['RegistrationYear']

data['SumInsured'] = pd.to_numeric(data['SumInsured'], errors='coerce')
data['CapitalOutstanding'] = pd.to_numeric(data['CapitalOutstanding'], errors='coerce')

# Create total capital
data['TotalCapital'] = data['SumInsured'] + data['CapitalOutstanding']


In [42]:
# One-hot encoding for categorical variables like Gender, Title, etc.
df = pd.get_dummies(data, columns=['Gender', 'Title', 'Language', 'Bank', 'AccountType', 'VehicleType'], drop_first=True)

In [43]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Define your features (X) and target (y)
X = df.drop(columns=['TotalPremium', 'TotalClaims'])  # Features excluding the target
y = df['TotalPremium']  # Target variable

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Drop datetime columns that cannot be directly used in the model
X_train = X_train.drop(columns=['TransactionMonth'], errors='ignore')
X_test = X_test.drop(columns=['TransactionMonth'], errors='ignore')



#### Linear Regression

In [ ]:
# Train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions
lr_predictions = lr_model.predict(X_test)

In [45]:
print(X_train.dtypes)

UnderwrittenCoverID                  int64
PolicyID                             int64
IsVATRegistered                       bool
Citizenship                         object
LegalType                           object
                                     ...  
AccountType_Transmission account      bool
VehicleType_Heavy Commercial          bool
VehicleType_Light Commercial          bool
VehicleType_Medium Commercial         bool
VehicleType_Passenger Vehicle         bool
Length: 69, dtype: object
